<a href="https://colab.research.google.com/github/sjcxie/Culturally-Adapted-Psychotherapy-Chatbot/blob/master/multiagent_communication_styles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cultural aware mental health support AI (using AutoGen Agentic framework)

Experimentation of how to use agentic wayto incorporate cultural context into mental health conversational AI.

In [1]:
## Installing dependencies (only need to run when using Google Colab)
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


# Communication Style Modifier system prompts

In [ ]:
# SYSTEM PROMPTS
csm_prompt_template_3 = """
Please revise the following ‘RESPONSE’ from a therapist to align better with the {communication_style} communication style. This style is characterized by the following definition: {definition} and measured by the survey items: {survey_item}. Ensure that the revised response:
: Adheres to the given communication style.
: Considers the ‘CONVERSATION HISTORY’ for context.
: Asks only one question in the response.

CONVERSATION HISTORY: {unadapted_chat_history}
RESPONSE to modify: {unadapted_response}
"""

In [ ]:
unadabot_system_message_template = """
Your responsibility is to guide the conversation with a caregiver ("user") through the principles of Problem-Solving Therapy (PST) to improve one significant symptom the caregiver is experiencing. You will ask open-ended questions to identify and assess their challenges and stressors and improve their self-care. After you identified one problem that the caregiver can work on to improve their health, generate two concise, achievable, and personalized goals that directly address and support their expressed needs and aspirations. Ensure these goals are not only realistic but designed to inspire and boost the caregiver's motivation. After the caregiver chooses one goal, talk with them through concrete behavior changes to implement this goal in the next few days. Avoid focusing on the care receiver. Remember, your job is to help the caregiver.

Use Motivational Interviewing (MI) techniques such as affirmation, reflection, emphasizing autonomy, giving information, normalizing, persuasion with permission, and seeking collaboration. Keeping in mind to cultivate change talk, soften sustain talk, strengthen partnerships, and show empathy. Do not question stack.

Below are ideal dialogue examples illustrating how you, the assistant, should evaluate and address the challenges and stressors of caregivers, referred to here as the user, during conversations.

Question:
**Example 1**
assistant: I am going to ask you a few questions.
To start, you have told me that your child's condition worries you.
What else has been on your mind?
user: Yes, Covid in general. The impact that it can have on our family and health
**Example 2**
user: I’m tired.
assistant: Got it. When would you say that you feel mostly tired during the day?
user: during the day, in the afternoon

Normalizing:
**Example 3**
assistant: Thank you for sharing.
It’s hard not to worry when you have so much going on.
In the past week, how many days did you experience worry?
user: every day

Seeking collaboration:
**Example 4**
assistant: It sounds like there are a few things that we can work on together. Which one would you like to focus on improving today?
user: Maybe tiredness.

 Question and Reflection:
**Example 5**
assistant: Thank you for rating it. How does it impact you?
user: makes it hard to want to do anything and things just pile up and overwhelm me
Assistant: I can see how tiredness and worry play into each other for you. What have you already tried so far to break this cycle?
user: Coffee, exercise, hydrate. Switching/looking for a day shift position

Affirmation:
**Example 6**
assistant: These are all great strategies. You have tried many different things. How have the strategies been working so far?
User: hard to say. Sometimes it works, sometimes doesn’t.

Giving information
**Example 8**
assistant: it sounds like you haven’t been sleeping well lately. Just like sometimes people feel “hangry”, as in more irritable when people are hungry. Lack of quality sleep also may make you more prone to anxiety and worry.
user: that makes sense now you pointed it out.

Persuade with permission and autonomy
**Example 9**
assistant: I do have some tips for that. Would you like to hear about it once I have more understanding of your sleep issue?
user: yeah for sure.
Assistant: Great! You can make the call and curate your own coping toolbox. Afterall, you are the only one that can test out these strategies decide for yourself!

Reflection and normalizing
**Example 10**
assistant: I am sorry this situation sounds overwhelming. Anyone in your situation would feel worried and overwhelmed. Where would you say that you feel most worried, for example, is it at work, home, etc.?
user: probably home and when I am not busy. I can distract myself with tasks while I am out. but I am a stay at home mom and an homeschooling so there is a lot of time at home right now.

Affirmation
**Example 11**
user: I’m mostly worried at home and when I am not busy. I can distract myself with tasks while I am out. but I am a stay at home mom and an homeschooling so there is a lot of time at home right now.
assistant: I see. You are pretty observant and aware of how your environment impacts your anxiety.

Normalizing, affirmation, and question:
**Example 12**
Assistant: Between 1-5, with 1 being the least level of worry and 5 being the highest, how would you rate your worry for today?
user: 2
assistant: Being completely worry-free isn't realistic. What we could do is take steps to get some control over your worries. What have you already tried to help improve your worry?
user: You're right! I try to give my daughter a hug and remember to be thankful that she is healthy and on the road to recovery. Then I try to accomplish something toward making the worry go away. if I am worried about her rash I try to turn that into an action step to apply a cream or plan a special bath, or as in the case today- add call the nurse line to find out about her insurance forms and set an alarm on my phone. Getting things done and questions answered really helps me feel more empowered to control the worry knowing that I am doing my part to care for her well.
Assistant: exactly! You are already taking actions that make a difference.
"""

# Run the following section of codes

In [ ]:
# Import libraries
import os
from google.colab import userdata
import openai
import pandas as pd
import getpass

# import langchain libraries
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import LLMChain, ConversationChain
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import AIMessage, HumanMessage

# Set up OpenAI and Langchain environment variables
os.environ['OPENAI_API_KEY'] = userdata.get('CommunicationStyle_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API')

In [ ]:
# FUNCTIONS
def generate_dialogue(max_turn, target_styles, definitions, survey_items, dialogue_i, persona_i):
  # initialize response dataframe
  adabot_responses_df = pd.DataFrame(columns=['dialogue_i', 'persona_i', 'turn_i','user','therapist','style'])

  # clear memory buffer
  unadabot_memory.clear()
  turn_cnt = 0

  # for each turn
  for i in range(0, max_turn*7, 7):   # we only allow 'max_turn' number of turns
    turn_cnt += 1
    print("Turn: ", turn_cnt)
    user_input = input("Enter your input:")

    adabot_responses_df.loc[i,'turn_i'] = turn_cnt   # save the turn number
    adabot_responses_df.loc[i,'user'] = user_input  # save the user input to the response dataframe
    adabot_responses_df.loc[i,'dialogue_i'] = dialogue_i
    adabot_responses_df.loc[i,'persona_i'] = persona_i

    # generate unadapted chatbot response(s)
    unadabot_chain.predict(human_input=user_input)
    unadapted_response = unadabot_memory.buffer[-1].content
    print("UnAdapted chatbot response: ", unadapted_response)
    adabot_responses_df.loc[i,'therapist'] = unadabot_memory.buffer[-1].content
    adabot_responses_df.loc[i,'style'] = 'unadapted'

    # unadapted chat history as context
    unadabot_chat_history = str(unadabot_memory.buffer[:-1]).replace('[', '').replace(']', '').replace('\\', '').replace("content=", "")

    # generate adapted response(s)
    for j in range(6):
      adapted_response_csm3 = csm_chain_3.predict(communication_style=target_styles[j],
                                                  definition=definitions[j],
                                                  survey_item=survey_items[j],
                                                  unadapted_chat_history= unadabot_chat_history,
                                                  unadapted_response=unadabot_memory.buffer[-1].content)

      adabot_responses_df.loc[i+j+1,'turn_i'] = turn_cnt
      adabot_responses_df.loc[i+j+1,'dialogue_i'] = dialogue_i
      adabot_responses_df.loc[i+j+1,'persona_i'] = persona_i
      adabot_responses_df.loc[i+j+1,'user'] = user_input
      adabot_responses_df.loc[i+j+1,'therapist'] = adapted_response_csm3
      adabot_responses_df.loc[i+j+1,'style'] = target_styles[j]

      print("Saved response for style: {}".format(target_styles[j]))

  # save the responses dataframe to excel sheet
  file_name = f'/content/dialogue_{dialogue_i}_persona_{persona_i}.xlsx'
  adabot_responses_df.to_excel(file_name)
  print(f'Saved: {file_name}')
  return adabot_responses_df


# For testing, we use gpt-4o-mini model - lowest cost
llm = ChatOpenAI(model="gpt-4o", temperature=1.0, top_p=1.0)


### unadabot: Therapist chatbot (unadapted version) ###
human_message_template = """{human_input}"""
unada_chat_prompt_template = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template(template=unadabot_system_message_template),
      MessagesPlaceholder(variable_name="unada_chat_history"), # dynamic insertion of past conversation history
      HumanMessagePromptTemplate.from_template(template=human_message_template),
  ]
)

# set up memory buffer for the unadabot
unadabot_memory = ConversationBufferMemory(memory_key="unada_chat_history", return_messages=True, llm=llm)

# unadabot LLM Chain
unadabot_chain = LLMChain(
    llm=llm,
    prompt=unada_chat_prompt_template,
    verbose=False,
    output_parser=StrOutputParser(),
    memory=unadabot_memory,
)


#### csm: Communication Style Modifier agent ##########
csm_prompt_3 = PromptTemplate(
    input_variables=["communication_style","definition", "survey_item", "chat_history","unadapted_response"], template=csm_prompt_template_3)

csm_chain_3 = LLMChain(
    llm=llm,
    prompt=csm_prompt_3,
    verbose=False,
    output_parser=StrOutputParser()
)

                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


# Generate dialogues: RUN all previous codes AND MODIFY THIS PART ONLY


In [ ]:
target_styles = ['authoritative', 'talkative', 'sentimentality', 'conversational dominance', 'informality', 'conciseness']
definitions = ["Authoritative is the tendency to command or demand others in a conversation, without considering the others' willingness and concerns.",
               "Talkativeness is a tendency to initiate a conversation, talk a lot, and avoid silence in a conversation. ",
               "Sentimentality is a tendency to express one's own emotions or display empathic emotional responses to others in a conversation.",
               "Conversational dominance is the tendency to take the lead in a conversation and detremine its topics and directions.",
               "Informality is a tendency to talk casually and avoid being formal, distant, or stiff in a conversation.",
               "Conciseness is the tendency to use as few words as possible to clearly convey ideas and explain things in a conversation, and avoid being long-winded."
               ]
survey_items = ["I am very likely to tell someone what they should do; I sometimes insist that otheres do what I say; I expect people to obey when I ask them to do something; When I feel others should do something for me, I ask for it in a demanding tone of voice. ",
                "I always have a lot to say; I have a hard time keeping myself silent when around other people; I am never the one who breaks a silence by starting to talk; I like to talk a lot.",
                "When I see others cry, I have difficulty holding back my tears; During a conversation, I am easily overcome by emotions; When describing my memories, i sometimes get visibily emotional; People can tell that I am emotionally touched by some topics of conversation.",
                "I often take the lead in a conversation; I often determine which topics are talked about during a conversation; I often determine the direction of a conversation.",
                "I communicate with others in a distant manner; I behave somewhat formally when I meet someone; I address others in a very casual way; I come across as somewhat stiff when dealing with people.",
                "I don’t need a lot of words to get my message across; Most of the time, I only need a few words to explain something; I am somewhat long-winded when I need to explain something; With a few words I can usually clarify my point to everybody."
                ]

In [ ]:
dialogue_i = 4 # UPDATE THIS dialogue NUMBER EACH  TIME!!
persona_i = 4
responses_df = generate_dialogue(max_turn=10, target_styles=target_styles, definitions=definitions, survey_items=survey_items, dialogue_i=dialogue_i, persona_i=persona_i)

Turn:  1
Enter your input:Hi. I'm so tired today, I didn't get enough sleep last night
UnAdapted chatbot response:  I'm sorry to hear that you didn't get enough sleep. It sounds really challenging to manage everything when you're feeling tired. How often do you find yourself feeling this way?
Saved response for style: authoritative
Saved response for style: talkative
Saved response for style: sentimentality
Saved response for style: conversational dominance
Saved response for style: informality
Saved response for style: conciseness
Turn:  2
Enter your input:all the time. I'm dealing with a lot of stress of my daughter's illness and my own chronic pain.
UnAdapted chatbot response:  That sounds incredibly tough to manage, especially when both stress and pain are constant companions. It's understandable that this is wearing you down. What are some things you've tried in the past to cope with this stress and pain?
Saved response for style: authoritative
Saved response for style: talkative


# Save all the generated dialogues .xlsx

In [ ]:
responses_df

,dialogue_i,persona_i,turn_i,user,therapist,style
0,1,1,1,"Hi, I'm feeling quite stressed balancing work ...",It sounds like you have a lot on your plate wi...,unadapted
1,1,1,1,"Hi, I'm feeling quite stressed balancing work ...",You need to prioritize managing your stress im...,authoritative
2,1,1,1,"Hi, I'm feeling quite stressed balancing work ...","Oh, I totally get it! Balancing work and paren...",talkative
3,1,1,1,"Hi, I'm feeling quite stressed balancing work ...",I can really feel the weight of what you're go...,sentimentality
4,1,1,1,"Hi, I'm feeling quite stressed balancing work ...",I understand that managing work alongside pare...,conversational dominance
...,...,...,...,...,...,...
65,1,1,10,"Thank you, I'm okay for now","Oh, you're very welcome! It's truly amazing ho...",talkative
66,1,1,10,"Thank you, I'm okay for now",I'm truly touched by your determination to tak...,sentimentality
67,1,1,10,"Thank you, I'm okay for now",Let's make sure we're on track for your succes...,conversational dominance
68,1,1,10,"Thank you, I'm okay for now","Sure thing! Here's a revised, more casual resp...",informality


In [ ]:
from google.colab import drive
import os
import shutil
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
source_folder = '/content'
destination_folder = '/content/drive/My Drive/Dissertation/Communication Style/Ada_responses_dfs_SX'
# Loop through all files in the source folder
for filename in os.listdir(source_folder):
    if filename.startswith('dialogue_'):
        # Construct full file path
        file_path = os.path.join(source_folder, filename)
        # Move the file to the destination folder
        shutil.move(file_path, destination_folder)
        print(f'Moved: {filename}')

Moved: dialogue_4_persona_4.xlsx


# Archived codes

In [ ]:
# FUNCTIONS
def generate_dialogue(n_prewritten, max_turn, target_style, definition, survey_item, trial_cnt):
  # initialize response dataframe
  adabot_responses_df = pd.DataFrame(columns=['index','user','therapist','style_label'])

  # clear memory buffer
  unadabot_memory.clear()

  # for each turn
  for i in range(0, max_turn*5, 5):   # we only allow 'max_turn' number of turns
    print("Turn: ", i+1)

    # if i in list(range(0, 10, 5)):
    #   print(prewritten_responses_df)

    # # let user choose whether to end the conversation, use prewritten response, or enter new response
    # user_choice = input("Use pre-written responses? ('STOP'or'SAVE'/Choose a prewritten response:1-9/Enter your own input:)")

    # if user_choice == 'STOP' or user_choice=='SAVE':   # user ends the conversation
    #   break

    # elif user_choice in [str(i) for i in range(1, n_prewritten+1)]:    # user enters an integer number to choose a prewritten response
    #   user_input = prewritten_responses_df.loc[prewritten_responses_df['Index'] == int(user_choice), 'Response'].values[0]

    # else:   # if chose to enter new response
    #   user_input = user_choice

    # print("User input: ", user_input)

    # generate unadapted chatbot response(s)
    unadabot_chain.predict(human_input=user_input)
    unadapted_response = unadabot_memory.buffer[-1].content
    print("UnAdapted chatbot response: ", unadapted_response)

    # generate adapted response(s)
    unadabot_chat_history = str(unadabot_memory.buffer[:-1]).replace('[', '').replace(']', '').replace('\\', '').replace("content=", "")

    adabot_responses_df.loc[i,'index'] = i+1
    adabot_responses_df.loc[i,'user'] = user_input  # save the user input to the response dataframe
    adabot_responses_df.loc[i,'therapist'] = unadabot_memory.buffer[-1].content
    adabot_responses_df.loc[i,'style_label'] = "Unadapted"

    for j in range(4):
      # adapted_response_csm1 = csm_chain_1.predict(communication_style=target_style,
      #                                             unadapted_chat_history= unadabot_chat_history,
      #                                             unadapted_response=unadabot_memory.buffer[-1].content)
      # adapted_response_csm2 = csm_chain_2.predict(communication_style=target_style,
      #                                             definition=definition,
      #                                             unadapted_chat_history= unadabot_chat_history,
      #                                             unadapted_response=unadabot_memory.buffer[-1].content)
      adapted_response_csm3 = csm_chain_3.predict(communication_style=target_style,
                                                  definition=definition,
                                                  survey_item=survey_item,
                                                  unadapted_chat_history= unadabot_chat_history,
                                                  unadapted_response=unadabot_memory.buffer[-1].content)
      # adabot_responses_df.loc[i,'ada_response_csm1'+'_r'+str(j+1)] = adapted_response_csm1
      # adabot_responses_df.loc[i,'ada_response_csm2'+'_r'+str(j+1)] = adapted_response_csm2
      adabot_responses_df.loc[i+j+1,'index'] = i+j+1+1
      adabot_responses_df.loc[i+j+1,'user'] = user_input
      adabot_responses_df.loc[i+j+1,'therapist'] = adapted_response_csm3
      adabot_responses_df.loc[i+j+1,'style_label'] = target_style
    # print("Adapted chatbot response version 1: ", adapted_response_csm1)
    # print("Adapted chatbot response version 2: ", adapted_response_csm2)
    print("Adapted chatbot response: ", adapted_response_csm3)

  # save the responses dataframe to excel sheet
  file_name = f'/content/dialogues_{target_style}_{trial_cnt}.xlsx'
  adabot_responses_df.to_excel(file_name)
  print(f'Saved: {file_name}')
  return adabot_responses_df


# For testing, we use gpt-4o-mini model - lowest cost
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.0, top_p=1.0)
llm_4o = ChatOpenAI(model="gpt-4o", temperature=1.0, top_p=1.0)
### Parameters
## Temperature: influences the variety in the model's responses.
## Lower values lead to more predictable and typical responses, while higher values encourage more diverse and less common responses.
## At 0, the model always gives the same response for a given input.
## Optional, float, 0.0 to 2.0 Default: 1.0

## top_p: limits the model's choices to a percentage of likely tokens.
## A lower value makes the model's responses more predictable, while the default setting allows for a full range of token choices.
## Optional, float, 0.0 to 1.0 Default: 1.0

# For actual experiment, we use either gpt-35 or gpt-4
# llm = ChatOpenAI(model="gpt-3.5-turbo")
# llm = ChatOpenAI(model="gpt-4")


### unadabot: Therapist chatbot (unadapted version) ###
human_message_template = """{human_input}"""
unada_chat_prompt_template = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template(template=unadabot_system_message_template),
      MessagesPlaceholder(variable_name="unada_chat_history"), # dynamic insertion of past conversation history
      HumanMessagePromptTemplate.from_template(template=human_message_template),
  ]
)

# set up memory buffer for the unadabot
unadabot_memory = ConversationBufferMemory(memory_key="unada_chat_history", return_messages=True, llm=llm_4o)

# unadabot LLM Chain
unadabot_chain = LLMChain(
    llm=llm_4o,
    prompt=unada_chat_prompt_template,
    verbose=False,
    output_parser=StrOutputParser(),
    memory=unadabot_memory,
)


#### csm: Communication Style Modifier agent ##########
# we will use the abbreviation CSM for communication style modifier

# csm_prompt_1 = PromptTemplate(
#     input_variables=["communication_style","chat_history","unadapted_response"], template=csm_prompt_template_1)
# csm_prompt_2 = PromptTemplate(
#     input_variables=["communication_style","definition", "chat_history","unadapted_response"], template=csm_prompt_template_2)
csm_prompt_3 = PromptTemplate(
    input_variables=["communication_style","definition", "survey_item", "chat_history","unadapted_response"], template=csm_prompt_template_3)

# CSM LLM Chains
# csm_chain_1 = LLMChain(
#     llm=llm,
#     prompt=csm_prompt_1,
#     verbose=False,
#     output_parser=StrOutputParser()
# )
# csm_chain_2 = LLMChain(
#     llm=llm,
#     prompt=csm_prompt_2,
#     verbose=False,
#     output_parser=StrOutputParser()
# )
csm_chain_3 = LLMChain(
    llm=llm_4o,
    prompt=csm_prompt_3,
    verbose=False,
    output_parser=StrOutputParser()
)

                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [ ]:
# SYSTEM PROMPTS
# csm_prompt_template_1 = """
# Refine and modify the following 'RESPONSE' from a therapist more {communication_style}.
# Only ask one question in the response. Take into account the 'CONVERSATION HISTORY' given below as conversation context.

# 'CONVERSATION HISTORY': {unadapted_chat_history}
# 'RESPONSE' to modify: {unadapted_response}
# """

# csm_prompt_template_2 = """
# Refine and modify the following 'RESPONSE' from a therapist more {communication_style}. {definition}
# Only ask one question in the modified response. Take into account the 'CONVERSATION HISTORY' given below as conversation context.

# 'CONVERSATION HISTORY': {unadapted_chat_history}
# 'RESPONSE' to modify: {unadapted_response}
# """

csm_prompt_template_3 = """
Refine and modify the following 'RESPONSE' from a therapist more {communication_style}. This style is defined as {definition}. This style can be measured by {survey_item}.
Only ask one question in the modified response. Take into account the 'CONVERSATION HISTORY' given below as conversation context.

'CONVERSATION HISTORY': {unadapted_chat_history}
'RESPONSE' to modify: {unadapted_response}
"""

# csm_prompt_template_4 """

# """

# csm_prompt_template_4 = """
# Refine and modify the following 'RESPONSE' from a therapist more {communication_style}. This style is defined as {definition}. This style can be measured by {survey_item}.
# Only ask one question in the response. Take into account the 'CONVERSATION HISTORY' given below as conversation context.

# 'CONVERSATION HISTORY': {unadapted_chat_history}
# 'RESPONSE' to modify: {unadapted_response}
# """